In [1]:
import geopandas as gpd
from scipy.spatial import KDTree
import numpy as np
from SimuSamp.functions.load.SpatDat import SpatDat
import pandas as pd
from SimuSamp.functions.compute.n_neighbours import neighbours

import matplotlib.pyplot as plt

In [2]:
parent_filepath = "../../../../Simulated WS Sampling/block_data"

sampleids = ["01_F", "01_G", "02_G", "02_H", "03_B", "03_H", "04_I", "06_H", "07_K", "08_J", "09_E", "10_B", "11_E", "11_F", "12_M", "13_G", "14_I", "15_G", "16_B", "17_I", "18_E", "19_F", "21_I", "22_N", "23_B", "24_I", "25_M", "26_I", "27_H", "28_FSC", "29_L", "30_D"]

microns_per_pixel=0.22715
mm2_per_pixels2 = (microns_per_pixel / 1000) ** 2

outpath_root = "../../../../Data/TMA_Spatial"

In [3]:
samples = []
coords = []
mean_nn_dists = []
for sampleid in sampleids:

    spatdat = SpatDat(sampleid, parent_filepath, tma=True)

    for coord in spatdat.core_map["Coords"]:

        cells, anno = spatdat.subset_cores(coord)

        anno = anno[anno["layer"] == "analysis_area"].reset_index(drop=True)

        # Plot cells and core annotation ===============================
        cells.plot(color="brown", markersize=5, alpha=0.75);
        anno.plot(ax=plt.gca(), facecolor="none", edgecolor="black", linewidth=2, linestyle="--");

        plt.title(f"{sampleid} - {coord}");

        plt.savefig(f"{outpath_root}/TMA_annotation_plots/{sampleid}_{coord}_annotation.png", dpi=300, bbox_inches="tight", transparent=True);

        plt.gca().cla();
        plt.clf();
        plt.close();
        # =============================================================

        # Calculate mean nearest-neighbour distance ===================
        nn_mean = neighbours(cells, 1, edge_correction=True, core=anno["geometry"], distance=600/microns_per_pixel)

        if isinstance(nn_mean, float):
            nn_mean = None
        else:
            nn_mean = nn_mean.mean() * microns_per_pixel

        samples.append(sampleid)
        coords.append(coord)
        mean_nn_dists.append(nn_mean)
        # =============================================================

nn_results = pd.DataFrame({"Sampleid": samples, "Coords": coords, "Mean_NN_Distance": mean_nn_dists})

nn_results.to_excel(f"{outpath_root}/TMA_mean_NN_distances.xlsx", index=False)

C:\Users\Michael\AppData\Local\Temp\ipykernel_22536\1018887159.py:15: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  cells.plot(color="brown", markersize=5, alpha=0.75);
